In [92]:
# Library Imports

from Bio.HMM.MarkovModel import HiddenMarkovModel, MarkovModelBuilder # Building the Markov Model
from Bio.Seq import Seq # Defining biological sequences through code
import numpy as np
import re

## Making a Basic Markov Model

The Markov Model was made to simulate the following image:
![example.png](example.png)

The transition states are Rainy and Sunny, while the emission states are "Walk", "Shop", and "Clean" (denoted as W, S, and C respectively in the code).

First, a MarkovModelBuilder was created using the aforentioned transition and emission probabilities. Then, a HiddenMarkovModel was built to run the Viterbi algorithm on it to determine the optimal hidden state configuration for a particular emission state combination.

In [35]:
# Dictionaries
transition_alphabet = ['Rainy', 'Sunny']
emission_alphabet = ['W', 'S', 'C'] # W - walk, S - shop, C - clean

# From the image above:
initial_prob = {'Rainy': 0.3, 'Sunny': 0.7}
transition_prob = {('Rainy', 'Rainy'): 0.4,
                 ('Rainy', 'Sunny'): 0.6,
                 ('Sunny', 'Rainy'): 0.2,
                 ('Sunny', 'Sunny'): 0.8}
emission_prob = {('Rainy', 'W'): 0.1,
                 ('Rainy', 'S'): 0.4,
                 ('Rainy', 'C'): 0.5,
                 ('Sunny', 'W'): 0.6,
                 ('Sunny', 'S'): 0.3,
                 ('Sunny', 'C'): 0.1}

In [36]:
# Markov Model Builder
build = MarkovModelBuilder(transition_alphabet, emission_alphabet)
build.set_initial_probabilities(initial_prob) # setting initial probabilities
build.allow_all_transitions()
for key in list(transition_prob.keys()):
    build.set_transition_score(key[0], key[1], transition_prob[key]) # adding transition probabilities
for key in list(emission_prob.keys()):
    build.set_emission_score(key[0], key[1], emission_prob[key]) # adding emission probabilities

In [37]:
# Initializing HiddenMarkovModel - last two parameters are pseudocounts
model = HiddenMarkovModel(transition_alphabet, emission_alphabet, initial_prob, transition_prob, emission_prob, 1, 1)

In [38]:
model.viterbi(Seq('WSC'), transition_alphabet)
# (Seq('SunnySunnyRainy'), -4.5972020163389145) - Sunny, Sunny, Rainy is the most optimal configuration with a 
# probability of 10^(-4.597)=0.00002529297 or 0.0025%

(Seq('SunnySunnyRainy'), -4.5972020163389145)

## Finding GC-Rich Regions

Next, this analysis of HMMs was translated to biology with attempting to differentiate background from promoter regions using the primary nucleotide sequences. This was the Hidden Markov Model represented:
![example2.png](example2.png)

Note the initial probabilities were defined as 0.5, because there is an equal chance of them being either a background or promoter region at the beginning.

In [39]:
transition_alphabet = ['B', 'P'] # B - background, P - promoter
emission_alphabet = ['A', 'T', 'C', 'G']
initial_prob = {'B': 0.5, 'P': 0.5}
transition_prob = {('B', 'B'): 0.85,
                  ('B', 'P'): 0.15,
                  ('P', 'P'): 0.75,
                  ('P', 'B'): 0.25}
emission_prob = {('B', 'A'): 0.25,
                ('B', 'T'): 0.25,
                ('B', 'C'): 0.25,
                ('B', 'G'): 0.25,
                ('P', 'A'): 0.15,
                ('P', 'T'): 0.13,
                ('P', 'C'): 0.42,
                ('P', 'G'): 0.3}

In [40]:
model = HiddenMarkovModel(transition_alphabet, emission_alphabet, initial_prob, transition_prob, emission_prob, 1, 1)

In [41]:
model.viterbi(Seq('AGTACACTGGT'), transition_alphabet)
# (Seq('BBBBBBBBBBB'), -17.567574447856494) - predicted no promoter regions

(Seq('BBBBBBBBBBB'), -17.567574447856494)

In [42]:
model.viterbi(Seq('GCAAATGC'), transition_alphabet)
# (Seq('BBBBBBBB'), -12.921134576003496) - predicted no promoter regions

(Seq('BBBBBBBB'), -12.921134576003496)

In [43]:
model.viterbi(Seq('GCGCGCGCAA'), transition_alphabet)
# (Seq('PPPPPPPPBB'), -15.314217188702496) - predicted promoter regions

(Seq('PPPPPPPPBB'), -15.314217188702496)

## HMMs with Memory (Dinucleotide Sequences)

Our last analysis looks at HMMS with memory where the value of the last nucleotide determines the value of the next nucleotide and what probability it is being selected. HMMs are usually memoryless, so the way to circumvent this is to define 8 transition states of A, C, G, T either being background (-) or promoter (+). Here was the Hidden Markov Model that was used:
![example3.png](example3.png)

Transition probabilities were taken from Durbin, Eddy et al. 1998. Note that because there is a 75% chance of going from promoter to promoter, 25% from going to promoter to background, 15% from going to background to promoter, and 85% from going from background to background, the probabilities have to be appropriately scaled. For example if the probability of an A+ going to an A is 0.18, then the probability of A+ going to A+ is 0.75x0.18 = 0.135, while the probability of A+ going to A- is 0.25x0.18 = 0.045. 

Initial probabilities were defined once again as equally likely to happen with 0.125.

In [44]:
def gen_transition_combs(val1, val2, val3, val4):
    return [(state1, state2) for state1 in transition_alphabet[val1:val2] for state2 in transition_alphabet[val3:val4]]

In [59]:
sequence = 'GCGCAAGAGCAT'
transition_alphabet = ['A+', 'C+', 'G+', 'T+', 'A-', 'C-', 'G-', 'T-']
emission_alphabet = ['A', 'T', 'C', 'G']

initial_prob = {key: 0.125 for key in transition_alphabet}
transition_prob = {}
emission_prob = {}

combs_pp = gen_transition_combs(0, 4, 0, 4)
combs_pb = gen_transition_combs(0, 4, 4, 8)
combs_bp = gen_transition_combs(4, 8, 0, 4)
combs_bb = gen_transition_combs(4, 8, 4, 8)
all_combs = combs_pp + combs_pb + combs_bp + combs_bb

# Transitions numbers from Durbin, Eddy et al. 1998
array_promoter = [0.18, 0.274, 0.426, 0.12, 0.171, 0.368, 0.274, 0.188, 0.161, 0.339, 0.375, 0.125, 0.079, 0.355, 0.384, 0.182]
vals_pp = [0.75*val for val in array_promoter]
vals_pb = [0.25*val for val in array_promoter]
array_background = [0.3, 0.205, 0.285, 0.21, 0.322, 0.298, 0.078, 0.302, 0.248, 0.246, 0.298, 0.208, 0.177, 0.239, 0.292, 0.292]
vals_bp = [0.15*val for val in array_background]
vals_bb = [0.85*val for val in array_background]
all_vals = vals_pp + vals_pb + vals_bp + vals_bb

for index in range(len(all_combs)):
    transition_prob[all_combs[index]] = all_vals[index]

for hidden_state in transition_alphabet:
    for emission_state in emission_alphabet:
        if (hidden_state[0] == emission_state):
            emission_prob[(hidden_state, emission_state)] = 1
        else:
            emission_prob[(hidden_state, emission_state)] = 0

In [60]:
model = HiddenMarkovModel(transition_alphabet, emission_alphabet, initial_prob, transition_prob, emission_prob, 1, 1)
model.viterbi(Seq(sequence), transition_alphabet)
# (Seq('G+C+G+C-A-A-T-G-A-C-A-G-T-C-A-G-T-G-C-G-A-G-C-A-T-'), -40.299037996610764) - detected promoter regions in the beginning

(Seq('G+C+G+C-A-A-G-A-G-C-A-T-'), -19.13769949224289)

## Viterbi Coding Implementation

In [78]:
def viterbi_algorithm(observations, states, start_p, trans_p, emit_p, table):
    V = [{}]
    
    # Initializes mechanism to store tracebacks and probabilities in future
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][observations[0]], "prev": None}
   
    for t in range(1, len(observations)):
        V.append({})
        for st in states:
            # Initialize - multiply by calculated probabilities of the last step and the transition probability for first state
            max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0] # Initialize variable
            for prev_st in states[1:]:
                # Find the maximum across all states by multiplying calculated probabilities by transition probability
                tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob # maximum probability for EACH state
                    prev_st_selected = prev_st
 
            # Multiply result by emission probability and store in V
            max_prob = max_tr_prob * emit_p[st][observations[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
    
    if table:
        for line in dptable(V):
            print(line)
 
    opt = []
    max_prob = 0.0
    best_st = None
 
    # Traceback and find maximum probability for last state
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st
 
    # Traceback to find optimal probabilities for all states - looking at "prev" to determine optimal pathway
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
 
    print (" ".join(opt) + ", probability = %s" % max_prob)

def dptable(V):
    yield " ".join(("%8d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.12s" % ("%f" % v[state]["prob"]) for v in V)

    
def structure(d):
    structured_d = {}
    for key in list(d.keys()):
        if (key[0] not in structured_d):
            structured_d[key[0]] = {key[1]: d[key]}
        else:
            structured_d[key[0]][key[1]] = d[key]
    return structured_d

In [79]:
viterbi_algorithm(list(sequence), transition_alphabet, initial_prob, structure(transition_prob), structure(emission_prob), True)

       0        1        2        3        4        5        6        7        8        9       10       11
A+: 0.000000 0.000000 0.000000 0.000000 0.000213 0.000029 0.000000 0.000001 0.000000 0.000000 0.000000 0.000000
C+: 0.000000 0.031781 0.000000 0.001661 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
G+: 0.125000 0.000000 0.006531 0.000000 0.000000 0.000000 0.000009 0.000000 0.000000 0.000000 0.000000 0.000000
T+: 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
A-: 0.000000 0.000000 0.000000 0.000000 0.000151 0.000039 0.000000 0.000002 0.000000 0.000000 0.000000 0.000000
C-: 0.000000 0.026137 0.000000 0.000554 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
G-: 0.125000 0.000000 0.002177 0.000000 0.000000 0.000000 0.000009 0.000000 0.000000 0.000000 0.000000 0.000000
T-: 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000

## Compare Viterbi Algorithms

In [100]:
print("HiddenMarkovModel Implementation")
print(model.viterbi(Seq(sequence), transition_alphabet)[0] + ", probability = " + str(model.viterbi(Seq(sequence), transition_alphabet)[1]))
print("\nOwn Implementation")
print(viterbi_algorithm(list(sequence), transition_alphabet, initial_prob, structure(transition_prob), structure(emission_prob), False))

HiddenMarkovModel Implementation
G+C+G+C-A-A-G-A-G-C-A-T-, probability = -19.13769949224289

Own Implementation
G+ C+ G+ C- A- A- G- A- G- C- A- T-, probability = 4.882055521997006e-09
None


In [98]:
re.split('[^a-zA-Z]', str(model.viterbi(Seq(sequence), transition_alphabet)[0]))

['G', 'C', 'G', 'C', 'A', 'A', 'G', 'A', 'G', 'C', 'A', 'T', '']